***Copyright 2024 by the authors' university which is unnamed because of KDD's anonymity requirement.***

##**Connect to Drive**

In [ ]:
from google.colab import drive
from google.colab import files

drive.mount('/content/gdrive/')
print("-"*80)

!ls "/content/gdrive/My Drive/Colab Notebooks/Research/CFE"

import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Research/CFE")
print("Current dir: ", os.getcwd())

##**Imports**

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings

from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, precision_recall_curve, roc_curve, auc
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

# Suppress warnings
warnings.filterwarnings("ignore")

##**Linear SVM**

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/CFE_datasets/income.csv')

df['income'] = df['income'].map({'>50K': 1, '<=50K': 0})

df = df.drop(columns=['fnlwgt', 'education'])
df.replace('?', pd.NA, inplace=True)
df = df.dropna()
df['sex'] = df['sex'].map({'Female': 1, 'Male': 0})

encoder = OrdinalEncoder()
df['workclass'] = encoder.fit_transform(df[['workclass']])
df['marital.status'] = encoder.fit_transform(df[['marital.status']])
df['occupation'] = encoder.fit_transform(df[['occupation']])
df['relationship'] = encoder.fit_transform(df[['relationship']])
df['race'] = encoder.fit_transform(df[['race']])
df['native.country'] = encoder.fit_transform(df[['native.country']])

X = df.loc[:, df.columns != 'income']
y = df['income']

In [ ]:
best = LinearSVC(C=1, random_state=42)
best.fit(X, y)
cv = StratifiedKFold(n_splits=10)
cv_scores = cross_val_score(best, X, y, cv=cv, scoring='f1_macro')

print("Cross-Validation Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())

train_predictions = best.predict(X)
train_accuracy = accuracy_score(y, train_predictions)
print("Training Set Accuracy:", train_accuracy)

Cross-Validation Scores: [0.54770499 0.69407496 0.61045253 0.61520417 0.67179082 0.59505796
 0.64485456 0.55199985 0.60220597 0.70000298]
Mean CV Accuracy: 0.623334878119963
Training Set Accuracy: 0.7510043630394402


##**Extracting weights and intercept values**

In [ ]:
for (intercept, coef) in zip(best.intercept_, best.coef_):
    s = "y = {0:.3f}".format(intercept)
    for (i, c) in enumerate(coef):
      s += " + {0} * x{1}".format(c, i)

    print(s)

y = -2.470 + 0.01810999346016009 * x0 + -0.029444571287443596 * x1 + 0.12631232059850578 * x2 + -0.07171255636860255 * x3 + -0.008154227262963876 * x4 + -0.038308107290164796 * x5 + 0.036613414256849004 * x6 + -0.27164581029474033 * x7 + -7.133555070207168e-05 * x8 + -0.00016082205350604565 * x9 + 0.003376782594382944 * x10 + -0.01582166868763174 * x11


In [ ]:
b = best.intercept_
w = best.coef_

w = w.flatten()

##**Setting the original misclassified *x* feature vector**

In [ ]:
og_x = df.drop(columns=['income'])
predicted_label = best.predict(og_x)

results_df = pd.DataFrame({'Income_Predicted': predicted_label})
index_class = results_df.index[results_df['Income_Predicted'] == 0]
og_x_reset = og_x.reset_index(drop=True)

filtered_records = og_x_reset.loc[index_class]
print(len(filtered_records))

##**Z3 implementation**

In [ ]:
!pip install z3
!pip install z3-solver

##**Optimizer -- step 1**

In [ ]:
import time
import csv
from z3 import *

def abs(x):
    return If(x < 0, -x, x)

def step1(W, B, og_x_np):
    n = W.shape[0]

    opt = Optimize()

    X = [Real(f'x_{i}') for i in range(n)]

    original_value = og_x_np[0]
    if original_value is not None:
        opt.add(X[0] == original_value)

    constraint1 = Or(X[1] == 1, X[1] == 2, X[1] == 3, X[1] == 4, X[1] == 5, X[1] == 6)
    opt.add(constraint1)

    original_value_ = og_x_np[2]
    if original_value_ is not None:
        opt.add(X[2] == original_value_)

    original_value3 = og_x_np[3]
    if original_value3 is not None:
        opt.add(X[3] == original_value3)

    constraint2 = Or(X[4] == 1, X[4] == 2,  X[4] == 3, X[4] == 4, X[4] == 5, X[4] == 6)
    opt.add(constraint2)

    original_value5 = og_x_np[5]
    if original_value5 is not None:
        opt.add(X[5] == original_value5)

    original_value6 = og_x_np[6]
    if original_value6 is not None:
        opt.add(X[6] == original_value6)

    original_value7 = og_x_np[7]
    if original_value7 is not None:
        opt.add(X[7] == original_value7)

    opt.add(And(X[8] >= 4000, X[8] <= 100000))

    opt.add(And(X[9] >= 1000, X[9] <= 2000))

    constraint3 = And(X[10] >= 20, X[10] <= 100)
    opt.add(constraint3)

    original_value11 = og_x_np[11]
    if original_value11 is not None:
        opt.add(X[11] == original_value11)

    dot_product = Sum([W[i] * X[i] for i in range(n)])

    inequality = dot_product + B > 0
    opt.add(inequality)

    l1_distance = Sum([abs(og_x_np[i] - X[i]) for i in range(n)])

    objective = l1_distance
    opt.minimize(objective)

    if opt.check() == sat:
        model = opt.model()

        original_f = [og_x_np[i] for i in range(n)]

        x_prime_optimized = [model[X[i]].as_decimal(15) for i in range(n)]
        print(f"Optimized x': {x_prime_optimized}")

        # Calculate the optimized distance
        l1_distance_1 = model.eval(objective).as_decimal(15)
        print(f"Optimized cost: {l1_distance_1}")

        cost_optimized = model.eval(objective).as_decimal(15)
        print(f"Optimized objective: {cost_optimized}")

        return original_f, x_prime_optimized, l1_distance_1, cost_optimized
    else:
        print("````````````FAILED TO FIND AN OPTIMAL SOLUTION.`````````````````````")
        original_f = [og_x_np[i] for i in range(n)]
        return [original_f, '', '', '']

W = w
B = b[0]
output_file = "/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_Household/Step1.csv"

for i in range(len(filtered_records)):
    row = filtered_records.iloc[i]
    og_x_np = row.to_numpy()

    start_time = time.time()

    original_f, x_prime_optimized, l1_distance_1, cost_optimized = step1(W, B, og_x_np)

    end_time = time.time()
    runtime = end_time - start_time

    with open(output_file, 'a', newline='') as file:
        writer = csv.writer(file)
        data_row = [original_f] + [x_prime_optimized] + [l1_distance_1] + [cost_optimized] + [runtime]
        writer.writerow(data_row)

In [ ]:
column_names = ['Original_x', 'Optimal_x', 'Cost', 'Objective', 'Runtime']
df1 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_Household/Step1.csv', names=column_names)

def remove_question_mark(cell_value):
    return cell_value.replace('?', '') if isinstance(cell_value, str) else cell_value

def remove_question_c(cell_value):
    return cell_value.replace('[', '') if isinstance(cell_value, str) else cell_value

def remove_question_cc(cell_value):
    return cell_value.replace(']', '') if isinstance(cell_value, str) else cell_value

def convert_to_float(lst_str):
    lst_str = str(lst_str)
    lst = lst_str.split(", ")
    return np.array([float(item.strip("'")) for item in lst])

cleaned_df = df1.applymap(remove_question_mark)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_c)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_cc)

cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].apply(convert_to_float)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_c)
cleaned_df['Optimal_x'] = cleaned_df['Optimal_x'].map(remove_question_cc)

optimized = cleaned_df.dropna()
left = cleaned_df[cleaned_df.isna().any(axis=1)]

optimized.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_Household/Step1_optimized.csv', index=False)
left.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_Household/Step1_failed.csv', index=False)

for i in range(len(optimized['Optimal_x'])):
    x = optimized['Optimal_x'].iloc[i]
    predicted_label = best.predict(x.reshape(1, -1))
    print(predicted_label)

In [ ]:
def step2(W, B, row):
    n = W.shape[0]
    opt = Optimize()
    X = [Real(f'x_{i}') for i in range(n)]

    original_value = row[0]
    if original_value is not None:
        opt.add(X[0] == original_value)

    constraint1 = Or(X[1] == 1, X[1] == 2, X[1] == 3, X[1] == 4, X[1] == 5, X[1] == 6)
    opt.add(constraint1)

    original_value_ = row[2]
    if original_value_ is not None:
        opt.add(X[2] == original_value_)

    original_value3 = row[3]
    if original_value3 is not None:
        opt.add(X[3] == original_value3)

    constraint2 = Or(X[4] == 1, X[4] == 2, X[4] == 3, X[4] == 4, X[4] == 5, X[4] == 6)
    opt.add(constraint2)

    original_value5 = row[5]
    if original_value5 is not None:
        opt.add(X[5] == original_value5)

    original_value6 = row[6]
    if original_value6 is not None:
        opt.add(X[6] == original_value6)

    original_value7 = row[7]
    if original_value7 is not None:
        opt.add(X[7] == original_value7)

    slack_cg = Real('capital_gain')
    constraint_cg = (slack_cg >= 0)
    opt.add(And(X[8]>0, X[8] >= 4000 - slack_cg, X[8] <= 100000 + slack_cg))
    opt.add(constraint_cg)

    slack_cl = Real('slack_cl')
    constraint_cl = (slack_cl >= 0)
    opt.add(And(X[9] > 0, X[9] >= 1000 - slack_cl, X[9] <= 2000 + slack_cl))
    opt.add(constraint_cl)

    slack_wh = Real('slack_wh')
    constraint_wh = (slack_wh >= 0)
    constraint3 = And(X[10] > 20 - slack_wh, X[10] <= 100 + slack_wh, X[10]<800)
    opt.add(constraint3, constraint_wh)

    original_value11 = row[11]
    if original_value11 is not None:
        opt.add(X[11] == original_value11)

    dot_product = Sum([W[i] * X[i] for i in range(n)])

    inequality = dot_product + B > 0
    opt.add(inequality)

    objective = (slack_wh + slack_cg + slack_cl)
    opt.minimize(objective)

    if opt.check() == sat:
        model = opt.model()

        original_f = [row[i] for i in range(n)]

        x_prime_optimized = [model[X[i]].as_decimal(15) for i in range(n)]
        print(f"Optimized x': {x_prime_optimized}")

        l1_distance_1 = model.eval(objective).as_decimal(15)
        print(f"Optimized cost: {l1_distance_1}")

        cost_optimized = model.eval(objective).as_decimal(15)
        print(f"Optimized objective: {cost_optimized}")

        slack_cg = model[slack_cg].as_decimal(15)
        print(f"Slack capital gain value: {slack_cg}")

        slack_wh = model[slack_wh].as_decimal(15)
        print(f"Slack workhours value: {slack_wh}")

        slack_cl = model[slack_cl].as_decimal(15)
        print(f"Slack capital loss value: {slack_cl}")

        return original_f, x_prime_optimized, l1_distance_1, cost_optimized, slack_cg, slack_cl, slack_wh
    else:
        print("````````````FAILED TO FIND AN OPTIMAL SOLUTION.`````````````````````")
        original_f = [row[i] for i in range(n)]
        return [original_f, '', '', '', '', '', '']

output_file1 = "/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_Household/Step2.csv"
input_file = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_Household/Step1_failed.csv')

input_file['Original_x'] = input_file['Original_x'].map(remove_question_c)
input_file['Original_x'] = input_file['Original_x'].map(remove_question_cc)

input_file['Original_x'] = input_file['Original_x'].apply(convert_to_float)

for i in range(len(input_file['Original_x'])):
    row = input_file['Original_x'].iloc[i]
    start_time = time.time()

    original_f, x_prime_optimized, cost, cost_optimized, slack_cg, slack_cl, slack_wh = step2(W, B, row)

    end_time = time.time()
    runtime = end_time - start_time

    with open(output_file1, 'a', newline='') as file:
        writer = csv.writer(file)
        data_row = [original_f] + [x_prime_optimized] + [cost] + [cost_optimized] + [runtime] + [slack_cg] + [slack_cl] + [slack_wh]
        writer.writerow(data_row)

In [ ]:
column_names = ['Original_x', 'Optimal_x', 'Cost', 'Objective', 'Runtime', 'Slack_cg', 'Slack_cl', 'Slack_wh']
df_step2 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_Household/Step2.csv', names = column_names)

df_step2 = df_step2.applymap(remove_question_mark)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_c)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_cc)

df_step2['Optimal_x'] = df_step2['Optimal_x'].apply(convert_to_float)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_c)
df_step2['Optimal_x'] = df_step2['Optimal_x'].map(remove_question_cc)

df_step2.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_Household/Step2.csv', index=False)

for i in range(len(df_step2['Optimal_x'])):
  x = df_step2['Optimal_x'].iloc[i]
  predicted_label = best.predict(x.reshape(1, -1))
  print(predicted_label)

In [ ]:
def step3(W, B, row, slack_value1, slack_value2, slack_value3):
    n = W.shape[0]

    opt = Optimize()

    X = [Real(f'x_{i}') for i in range(n)]

    original_value = row[0]
    if original_value is not None:
        opt.add(X[0] == original_value)

    constraint1 = Or(X[1] == 1, X[1] == 2, X[1] == 3, X[1] == 4, X[1] == 5, X[1] == 6)
    opt.add(constraint1)

    original_value_ = row[2]
    if original_value_ is not None:
        opt.add(X[2] == original_value_)

    original_value3 = row[3]
    if original_value3 is not None:
        opt.add(X[3] == original_value3)

    constraint2 = Or(X[4] == 1, X[4] == 2, X[4] == 3, X[4] == 4, X[4] == 5, X[4] == 6)
    opt.add(constraint2)

    original_value5 = row[5]
    if original_value5 is not None:
        opt.add(X[5] == original_value5)

    original_value6 = row[6]
    if original_value6 is not None:
        opt.add(X[6] == original_value6)

    original_value7 = row[7]
    if original_value7 is not None:
        opt.add(X[7] == original_value7)

    opt.add(And(X[8] >= 4000 - slack_value1, X[8] <= 100000 + slack_value1))

    opt.add(And(X[9] >= 1000 - slack_value2, X[9] <= 2000 + slack_value2))

    constraint3 = And(X[10] >= 20 - slack_value3, X[10] <= 100 + slack_value3)
    opt.add(constraint3)

    original_value11 = row[11]
    if original_value11 is not None:
        opt.add(X[11] == original_value11)

    dot_product = Sum([W[i] * X[i] for i in range(n)])

    inequality = dot_product + B > 0
    opt.add(inequality)

    l0 = Sum([If(row[i] != X[i], 1, 0) for i in range(n)])

    objective = l0
    opt.minimize(objective)

    if opt.check() == sat:
        model = opt.model()

        original_f = [row[i] for i in range(n)]

        x_prime_optimized = [model[X[i]].as_decimal(15) for i in range(n)]
        print(f"Optimized x': {x_prime_optimized}")

        l1_distance_1 = model.eval(objective)
        print(f"Optimized distance: {l1_distance_1}")

        cost_optimized = model.eval(objective)
        print(f"Optimized cost: {cost_optimized}")

        return original_f, x_prime_optimized, l1_distance_1, cost_optimized
    else:
        print("````````````FAILED TO FIND AN OPTIMAL SOLUTION.`````````````````````")
        original_f = [row[i] for i in range(n)]
        return [original_f, '', '', '']

out = '/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_Household/Step3.csv'
last = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_Household/Step2.csv')

last['Original_x'] = last['Original_x'].map(remove_question_c)
last['Original_x'] = last['Original_x'].map(remove_question_cc)
last['Original_x'] = last['Original_x'].apply(convert_to_float)

for i in range(len(last['Original_x'])):
    row = last['Original_x'].iloc[i]
    slack_value1 = last['Slack_cg'].iloc[i]
    slack_value2 = last['Slack_cl'].iloc[i]
    slack_value3 = last['Slack_wh'].iloc[i]
    start_time = time.time()

    original_f, x_prime_optimized, l1_distance_1, cost_optimized = step3(W, B, row, slack_value1, slack_value2, slack_value3)

    end_time = time.time()
    runtime = end_time - start_time

    with open(out, 'a', newline='') as file:
        writer = csv.writer(file)
        data_row = [original_f] + [x_prime_optimized] + [l1_distance_1] + [cost_optimized] + [runtime]
        writer.writerow(data_row)

In [ ]:
column_names = ['Original_x', 'Optimal_x', 'Cost', 'Objective', 'Runtime']
df_step3 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_Household/Step3.csv', names = column_names)

df_step3 = df_step3.applymap(remove_question_mark)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_c)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_cc)

df_step3['Optimal_x'] = df_step3['Optimal_x'].apply(convert_to_float)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_c)
df_step3['Optimal_x'] = df_step3['Optimal_x'].map(remove_question_cc)

df_step3.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_Household/Step3.csv', index=False)

for i in range(len(df_step3['Optimal_x'])):
  x = df_step3['Optimal_x'].iloc[i]
  predicted_label = best.predict(x.reshape(1, -1))
  print(predicted_label)

## **Average runtimes for l0 and l1**

In [ ]:
final_l0 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_Household/Step3.csv')
l0_runtime = final_l0.loc[:, 'Runtime'].mean()

print('L0 runtime', l0_runtime)

In [ ]:
d3_initial = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_Household/Step1_failed.csv')
d1 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_Household/Step1_optimized.csv')
d = pd.concat([d1, d3_initial])
l1_runtime = d.loc[:, 'Runtime'].mean()
print('L1 runtime', l1_runtime)

## **Percentage of relaxations**

In [ ]:
relaxed_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_Household/Step2.csv')
needed_relaxing = len(relaxed_df)

subset_of_experiment = len(filtered_records)

percentage_of_relaxations = (needed_relaxing/subset_of_experiment)*100
print(percentage_of_relaxations)

### **Average percentage of relaxed soft constraints**

In [ ]:
relaxed_df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Research/CFE/Public/ImplementationResults/American_Household/Step2.csv')
number_of_slacks_per_record = 3

count_s1 = 0
count_s2 = 0
count_s3 = 0
total_count = 0
total_sum = 0
for i in range(len(relaxed_df)):
    if relaxed_df['Slack_cg'].iloc[i] != 0:
        count_s1 = 1
    elif relaxed_df['Slack_cg'].iloc[i] == 0:
        count_s1 = 0
    if relaxed_df['Slack_cl'].iloc[i] != 0:
        count_s2 = 1
    elif relaxed_df['Slack_cl'].iloc[i] == 0:
        count_s2 = 0
    if relaxed_df['Slack_wh'].iloc[i] != 0:
        count_s3 = 1
    elif relaxed_df['Slack_wh'].iloc[i] == 0:
        count_s3 = 0
    total_count = count_s1 + count_s2 + count_s3
    total_sum += (total_count / number_of_slacks_per_record)

average_per_dataset = total_sum / len(relaxed_df) * 100

print(average_per_dataset)